In [1]:
# Indian Stock Market Reinvestment Advisor (Unified Script)
# Modes:
#   1) Single Company (deep dive + recommendation + FV)
#   2) Multi-Company (rank by Decision Score + FV per company)

import warnings
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import yfinance as yf

warnings.filterwarnings("ignore", category=FutureWarning)

START_DATE = (datetime.today() - timedelta(days=365 * 10)).strftime('%Y-%m-%d')
END_DATE = datetime.today().strftime('%Y-%m-%d')
RISK_FREE_RATE = 0.05  # annual risk-free rate used in Sharpe/Sortino

def _safe_series_from_df(df: pd.DataFrame) -> pd.Series:
    """Pick a reasonable price series from a yfinance DataFrame."""
    if "Adj Close" in df.columns:
        return df["Adj Close"].dropna()
    if "Close" in df.columns:
        return df["Close"].dropna()
    num = df.select_dtypes("number")
    if not num.empty:
        return num.iloc[:, 0].dropna()
    return pd.Series(dtype=float)

def download_stock_data(ticker: str) -> pd.Series:
    """Download 10-year adjusted/close data for a single NSE company."""
    sym = ticker if ticker.endswith(".NS") else ticker + ".NS"
    try:
        df = yf.download(sym, start=START_DATE, end=END_DATE, progress=False)
        price = _safe_series_from_df(df)
        return price
    except Exception:
        return pd.Series(dtype=float)

def compute_cagr(series: pd.Series) -> float:
    s = series.dropna()
    if s.empty:
        return np.nan
    years = (s.index[-1] - s.index[0]).days / 365.25
    return float((s.iloc[-1] / s.iloc[0]) ** (1 / years) - 1)

def downside_std(returns: pd.Series) -> float:
    neg = returns[returns < 0]
    return float(neg.std()) if not neg.empty else 0.0

def rolling_recovery_days(price: pd.Series) -> float:
    s = price.dropna()
    if s.empty:
        return np.nan
    peak = s.iloc[0]
    last_peak_dt = s.index[0]
    max_days = 0
    for dt, val in s.items():
        if val >= peak:
            peak = val
            last_peak_dt = dt
        else:
            days_under = (dt - last_peak_dt).days
            if days_under > max_days:
                max_days = days_under
    return float(max_days)

def calculate_metrics(price: pd.Series) -> dict:
    """Compute core performance & risk metrics for one stock."""
    if isinstance(price, pd.DataFrame):
        price = price.iloc[:, 0]
    returns = price.pct_change().dropna()

    # Market reference for beta
    ref_df = yf.download("^NSEI", start=START_DATE, end=END_DATE, progress=False)
    nifty_series = _safe_series_from_df(ref_df)
    nifty_ret = nifty_series.pct_change().dropna()

    cagr = compute_cagr(price)
    ann_vol = float(returns.std() * np.sqrt(252))
    ann_ret = float(returns.mean() * 252)

    sharpe = float((ann_ret - RISK_FREE_RATE) / ann_vol) if ann_vol > 0 else np.nan
    ddown = downside_std(returns) * np.sqrt(252)
    sortino = float((ann_ret - RISK_FREE_RATE) / ddown) if ddown > 0 else np.nan

    roll_max = price.cummax()
    dd = (price / roll_max) - 1
    max_dd = float(dd.min()) if not dd.empty else np.nan
    calmar = float(ann_ret / abs(max_dd)) if (max_dd is not None and max_dd < 0) else np.nan

    common = returns.index.intersection(nifty_ret.index)
    aligned = pd.concat([returns.loc[common], nifty_ret.loc[common]], axis=1).dropna()
    if len(aligned) > 100 and np.var(aligned.iloc[:, 1]) > 0:
        beta = float(np.cov(aligned.iloc[:, 0], aligned.iloc[:, 1])[0, 1] / np.var(aligned.iloc[:, 1]))
    else:
        beta = np.nan

    rec_days = rolling_recovery_days(price)

    return {
        "CAGR_10Y": cagr,
        "Sharpe": sharpe,
        "Sortino": sortino,
        "Calmar": calmar,
        "Volatility": ann_vol,
        "Max_Drawdown": max_dd,
        "Beta": beta,
        "Recovery_Days": rec_days
    }

def project_future_value(amount: float, cagr: float, years: int) -> float:
    try:
        return amount * ((1 + cagr) ** years)
    except Exception:
        return np.nan

def generate_recommendation(metrics: dict) -> str:
    """Simple rules for single-company reinvestment."""
    cagr = metrics.get("CAGR_10Y", np.nan)
    sharpe = metrics.get("Sharpe", np.nan)
    vol = metrics.get("Volatility", np.nan)

    if (pd.notna(sharpe) and pd.notna(cagr) and pd.notna(vol)
        and sharpe > 0.5 and cagr > 0.12 and vol < 0.35):
        return "✅ Strong reinvestment opportunity — steady growth and moderate risk."
    elif (pd.notna(sharpe) and pd.notna(cagr)
          and 0.3 < sharpe <= 0.5 and 0.08 < cagr <= 0.12):
        return "⚠️ Moderate potential — consider partial reinvestment; returns steady but risk slightly higher."
    else:
        return "❌ Risky reinvestment — performance inconsistent or below market average."

# ---------- Multi-company scoring ----------
def _norm(v, cap): return max(0, min(v / cap, 1)) if pd.notna(v) else 0
def _inv(v, cap): return max(0, min(1 - v / cap, 1)) if pd.notna(v) else 0
def _dd_norm(v): return max(0, min(1 - abs(v) / 0.6, 1)) if pd.notna(v) else 0
def _beta_norm(v):
    if pd.isna(v): return 0.5
    v = float(v)
    if v <= 1.2: return 1 - abs(v - 1) / 0.4
    return max(0, 1 - (v - 1.2) / 0.8)

WEIGHTS = {
    "CAGR_10Y": 25, "Sharpe": 15, "Sortino": 10, "Calmar": 10,
    "Volatility": 10, "Max_Drawdown": 10, "Beta": 5, "Recovery_Days": 5
}

def score_stock(metrics: dict) -> float:
    s = 0
    s += _norm(metrics.get("CAGR_10Y"), 0.20) * WEIGHTS["CAGR_10Y"]
    s += _norm(metrics.get("Sharpe"), 1.5) * WEIGHTS["Sharpe"]
    s += _norm(metrics.get("Sortino"), 2.0) * WEIGHTS["Sortino"]
    s += _norm(metrics.get("Calmar"), 1.5) * WEIGHTS["Calmar"]
    s += _inv(metrics.get("Volatility"), 0.35) * WEIGHTS["Volatility"]
    s += _dd_norm(metrics.get("Max_Drawdown")) * WEIGHTS["Max_Drawdown"]
    s += _beta_norm(metrics.get("Beta")) * WEIGHTS["Beta"]
    s += _inv(metrics.get("Recovery_Days"), 800) * WEIGHTS["Recovery_Days"]
    return round((s / sum(WEIGHTS.values())) * 100, 2)

# INTERACTIVE MODE SELECTOR
print("\n🔀 Reinvestment Mode Options:")
print("1) SINGLE  — Analyze one company you already own")
print("2) MULTI   — Compare multiple companies you own")

try:
    mode = input("\nChoose mode (SINGLE/MULTI) [default SINGLE]: ").strip().upper() or "SINGLE"
except Exception:
    print("\nNotebook input not available; defaulting to SINGLE.")
    mode = "SINGLE"

# SINGLE COMPANY MODE
if mode == "SINGLE":
    try:
        company = input("\n🏢 Enter the company ticker (e.g., INFY, TCS, HDFCBANK): ").strip().upper()
    except Exception:
        print("\nNotebook mode detected. Defaulting to INFY.")
        company = "INFY"

    try:
        years = int(input("📆 Reinvestment duration in years (default 5): ") or 5)
    except Exception:
        years = 5

    try:
        amount = float(input("💰 Reinvestment amount in ₹ (default 100000): ") or 100000)
    except Exception:
        amount = 100000

    price = download_stock_data(company)
    if price.empty:
        print(f"\n⚠️ Unable to retrieve data for {company}. Please check the ticker and try again.")
    else:
        m = calculate_metrics(price)
        fv = project_future_value(amount, m["CAGR_10Y"], years)
        reco = generate_recommendation(m)

        print(f"\n✅ Reinvestment Summary for {company} ({years}-Year Outlook)")
        print("-------------------------------------------------------------")
        print(f"💰 CAGR (10Y): {m['CAGR_10Y']*100:,.2f}%")
        print(f"📈 Sharpe Ratio: {m['Sharpe']:.2f}")
        print(f"📉 Sortino Ratio: {m['Sortino']:.2f}")
        print(f"📊 Calmar Ratio: {m['Calmar']:.2f}")
        print(f"🛡️ Volatility: {m['Volatility']*100:,.2f}%")
        print(f"📉 Max Drawdown: {m['Max_Drawdown']*100:,.2f}%")
        print(f"📉 Beta vs NIFTY: {m['Beta']:.2f}")
        print(f"🕒 Recovery Days: {m['Recovery_Days']:.0f} days")

        print("\n💵 Future Projection:")
        print(f"If you reinvest ₹{amount:,.0f} for {years} years → Expected Value ≈ ₹{fv:,.0f}")

        print("\n💡 Recommendation:")
        print(reco)

        print("\n🧾 Explanation of Terms:")
        print("• CAGR: Average annual growth rate over 10 years — shows long-term compounding power.")
        print("• Sharpe: Return vs total risk. Higher (>0.5) = steadier performance.")
        print("• Sortino: Return vs downside risk (ignores good volatility). Higher = safer.")
        print("• Calmar: Return relative to worst drawdown. Higher = better recovery.")
        print("• Volatility: Price fluctuation level. Lower = more stable.")
        print("• Max Drawdown: Biggest fall from peak. Lower = safer.")
        print("• Beta: Sensitivity to market. <1 = less volatile than NIFTY.")
        print("• Recovery Days: Time to bounce back from worst dip. Fewer = better.")

# MULTI-COMPANY MODE
elif mode == "MULTI":
    print("\n💡 Enter multiple tickers separated by commas (e.g., INFY, TCS, RELIANCE, HDFCBANK)")
    try:
        tickers = input("🏢 Companies: ").strip().upper().replace(" ", "").split(",")
    except Exception:
        print("\nNotebook mode detected. Using default sample: INFY,TCS,HDFCBANK")
        tickers = ["INFY", "TCS", "HDFCBANK"]

    try:
        years = int(input("📆 Reinvestment duration in years (default 5): ") or 5)
    except Exception:
        years = 5

    try:
        amount = float(input("💰 Reinvestment amount per company in ₹ (default 100000): ") or 100000)
    except Exception:
        amount = 100000

    results = []
    for t in tickers:
        price = download_stock_data(t)
        if price.empty:
            print(f"⚠️ Skipping {t}: No data found.")
            continue
        met = calculate_metrics(price)
        score = score_stock(met)
        fv = project_future_value(amount, met["CAGR_10Y"], years)
        met.update({"Company": t, "Decision_Score": score, "Future_Value": fv})
        results.append(met)

    if not results:
        print("\n⚠️ No valid data for any entered company.")
    else:
        df = pd.DataFrame(results).set_index("Company").sort_values("Decision_Score", ascending=False)

        print("\n✅ Reinvestment Analysis Complete!")
        print("🏆 Ranking of Companies (Based on Safety + Growth Potential):\n")
        for i, (sym, row) in enumerate(df.iterrows(), start=1):
            print(f"{i}. {sym} → Score: {row['Decision_Score']:.2f}/100 | "
                  f"CAGR: {row['CAGR_10Y']*100:,.2f}% | Sharpe: {row['Sharpe']:.2f}")

        print("\n💰 Estimated Future Value per ₹{:,} invested for {} years:".format(int(amount), years))
        for sym, row in df.iterrows():
            print(f"• {sym}: ₹{row['Future_Value']:,.0f}")

        print("\n🧭 Recommendation Summary:")
        print("→ Top 1–2: Ideal for increased reinvestment (best balance of risk & growth).")
        print("→ Middle scores: Maintain moderate exposure.")
        print("→ Low scores (<40): Risky or inconsistent; avoid fresh reinvestment.")

        print("\n🧾 Metric Explanation:")
        print("• CAGR – average annual growth rate (long-term performance).")
        print("• Sharpe – return per unit of total risk (higher = steadier).")
        print("• Sortino – return per unit of downside risk (ignores good volatility).")
        print("• Calmar – annual return / biggest loss (recovery ability).")
        print("• Volatility – degree of price fluctuation (lower = stable).")
        print("• Max Drawdown – biggest fall from peak (lower = safer).")
        print("• Beta – sensitivity to market (1 = market-level risk; <1 = safer).")
        print("• Recovery Days – time to recover from worst dip (lower = resilient).")

# INVALID MODE
else:
    print("\n⚠️ Invalid choice. Please run again and choose SINGLE or MULTI.")



🔀 Reinvestment Mode Options:
1) SINGLE  — Analyze one company you already own
2) MULTI   — Compare multiple companies you own

✅ Reinvestment Summary for INFY (5-Year Outlook)
-------------------------------------------------------------
💰 CAGR (10Y): 14.54%
📈 Sharpe Ratio: 0.47
📉 Sortino Ratio: 0.64
📊 Calmar Ratio: 0.47
🛡️ Volatility: 26.32%
📉 Max Drawdown: -36.55%
📉 Beta vs NIFTY: 0.83
🕒 Recovery Days: 917 days

💵 Future Projection:
If you reinvest ₹1,000 for 5 years → Expected Value ≈ ₹1,972

💡 Recommendation:
❌ Risky reinvestment — performance inconsistent or below market average.

🧾 Explanation of Terms:
• CAGR: Average annual growth rate over 10 years — shows long-term compounding power.
• Sharpe: Return vs total risk. Higher (>0.5) = steadier performance.
• Sortino: Return vs downside risk (ignores good volatility). Higher = safer.
• Calmar: Return relative to worst drawdown. Higher = better recovery.
• Volatility: Price fluctuation level. Lower = more stable.
• Max Drawdown: B